In [1]:
import numpy as np
import pandas as pd


In [4]:
columns_names =["user_id","item_id","rating","timestamp"]
df = pd.read_csv("u.data",sep="\t",names= columns_names)

In [5]:
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [6]:
movie_t = pd.read_csv("Movie_Id_Titles")

In [7]:
movie_t.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [8]:
df = pd.merge(df,movie_t,on="item_id")

In [9]:
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [18]:
n_users = df.user_id.nunique()


In [19]:
n_items = df.item_id.nunique()

In [24]:
from sklearn.model_selection import train_test_split

In [17]:
train_data, test_data = train_test_split(df,test_size=0.25)

In [25]:
train_data_matriz = np.zeros((n_users,n_items))
for line in train_data.itertuples():
    train_data_matriz[line[1]-1,line[2]-1] = line[3]

test_data_matriz = np.zeros((n_users,n_items))
for line in test_data.itertuples():
    test_data_matriz[line[1]-1,line[2]-1] = line[3]

In [26]:
train_data_matriz

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [27]:
from sklearn.metrics.pairwise import pairwise_distances 

In [28]:
user_similarity = pairwise_distances(train_data_matriz,metric="cosine")
item_similarity = pairwise_distances(train_data_matriz.T,metric="cosine")

In [29]:
train_data_matriz.T

array([[5., 4., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 5., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
train_data_matriz

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [33]:
item_prediction = predict(train_data_matriz, item_similarity, type='item')
user_prediction = predict(train_data_matriz, user_similarity, type='user')

In [34]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [36]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matriz)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matriz)))

User-based CF RMSE: 3.1230258883003796
Item-based CF RMSE: 3.448262273865393
